In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Creating Legal Entities and Instrument Relationships

This notebook demonstrates how you can create legal entities with multiple identifiers from different sources and utilise legal entity-instruments relationships to filter a universe of instruments based on the properties attached to the legal entities.

Attributes
----------
legal entities
relationships
"""

toggle_code("Toggle Docstring")

# Creating Legal Entities and Instrument Relationships
This notebook demonstrates that with LUSID, you can collate information from various sources onto one legal entity and use that collection of data to inform your decisions for creating an investable universe of instruments. We will:

 - Add multiple identifiers from different sources to one legal entity (LE) by merging multiple legal entity data sources.
 - Enrich legal entities with 2 different sources of Diversity, Equity and Inclusion data
 - Link instruments to their legal entities using legal entity-instrument relationships
 - Use instrument identifers to query properties on legal entities via the LE-instrument relationships created
 - Create a universe of instruments filtered by the DEI data stored as properties on the instruments' legal entities

In [2]:
import os
import pandas as pd
import datetime
import json
import pytz
from datetime import datetime
from IPython.core.display import HTML

# Then import the key modules from the LUSID package (i.e. The LUSID SDK)
import lusid as lu
import lusid.api as la
import lusid.models as lm
import logging
logging.basicConfig(level=logging.INFO)

from lusidtools import cocoon as lpt

# And use absolute imports to import key functions from Lusid-Python-Tools and other helper package

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.jupyter_tools import StopExecution
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
)

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path=os.getenv("FBN_SECRETS_PATH")

if secrets_path is None:
    secrets_path=os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory=ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status=pd.DataFrame(
    api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.10419.0,0.5.3112,"{'relation': 'RequestLogs', 'href': 'http://fb..."


In [3]:
relationship_definitions_api = api_factory.build(lu.RelationshipDefinitionsApi)
relationships_api = api_factory.build(lu.RelationshipsApi)
property_definitions_api = api_factory.build(lu.PropertyDefinitionsApi)
legal_entities_api = api_factory.build(lu.LegalEntitiesApi)
instruments_api = api_factory.build(lu.InstrumentsApi)
search_api = api_factory.build(lu.SearchApi)

In [4]:
id_scope = "legalEntityRelationships" # acts as main scope for legal entities
dei_scopeA = "deiDataSourceA"
dei_scopeB = "deiDataSourceB"

### Load a DataFrame of Legal Entity Identifiers (LEIs) for UK equities.

In [5]:
df_legal_entities=pd.read_csv("data/legal_entities.csv")
df_legal_entities.head()

,entityName,LEI
0,BAE Systems PLC,8SVCSVKSGDWMW2QHOH83
1,Phoenix Group Holdings PLC,2138001P49OLAEU33T68
2,WPP PLC,549300LSGBXPYHXGDT93
3,United Utilities Group PLC,2138002IEYQAOC88ZJ59
4,Smith & Nephew PLC,213800ZTMDN8S67S1H61


A legal entity is represented as a collection of properties. Identifier properties are defined separately and used to find the legal entity while other properties are used to attach more information to the legal entity.

## Create identifier property
First we must create the identifier property for the Legal Entity. Set domain to "LegalEntity" and constraint style to "Identifier" so lusid knows it is an identifier property for a legal entity.


In [6]:
# create identifier property definition

def create_property_identifier(code, display_name):
    try:
        property_definitions_api.create_property_definition(
            create_property_definition_request=lm.CreatePropertyDefinitionRequest(
                domain="LegalEntity",
                scope=id_scope,
                code=code,
                value_required=None,
                display_name=display_name,
                data_type_id=lm.ResourceId(scope="system", code="string"),
                life_time="Perpetual",
                constraint_style="Identifier"
            )
        )
    except:
        print("Property already exists")

This identifier property will be the standard Legal Entity Identifer (LEI) from the DataFrame we loaded.

In [7]:
create_property_identifier("LEI", "Legal Entity Identifier")

Property already exists


## Create Legal Entities

In [8]:
def create_legal_entity(identifier_property, LEI, LE_name):

    legal_entity_request=lm.UpsertLegalEntityRequest(
        identifiers={f"LegalEntity/{id_scope}/{identifier_property}":lm.PerpetualProperty(
                        key=f"LegalEntity/{id_scope}/{identifier_property}",
                        value=lm.PropertyValue(label_value=LEI)
                        ),
                    },

        display_name=LE_name,
        description=LE_name,
        counterparty_risk_information=None,
        )

    legal_entities_api.upsert_legal_entity(legal_entity_request)

In [9]:
legal_entities = []

for i in enumerate(df_legal_entities["LEI"]):
    identifier_property="LEI"
    LEI=df_legal_entities["LEI"][i[0]]
    LE_name=df_legal_entities["entityName"][i[0]]
    
    create_legal_entity(identifier_property, LEI, LE_name)
    legal_entities.append(LEI)
    
print(len(legal_entities), "legal entities created")

102 legal entities created


#### Add non-standard identidiers from another source

The next step is to add another identifier to the legal entities we just created as some data sources may have custom symbols in addition to the standard LEI. Here, we are adding a unique ID Number from another data source. This is also one of the datasets we will use to add diversity data to our properties later on in this notebook.

In [10]:
# Load a DataFrame of Diversity, Equity and Inclusion (DEI) data
df_dei_sourceA=pd.read_csv("data/dei_data_sourceA.csv")
df_dei_sourceA.head()

,Name,Custom Legal Entity Identifier,Legal Entity Identifier,Total DEI Score,Total Gender Score
0,IMPERIAL BRANDS PLC,944941,549300DFVPOB67JL3A42,40.62,35.49
1,PHOENIX GROUP HOLDINGS PLC,1567007,2138001P49OLAEU33T68,33.20,25.58
2,ADMIRAL GROUP PLC,312839,213800FGVM7Z9EJB2685,43.23,69.07
3,ANGLO AMERICAN PLC,1190816,549300S9XF92D1X8ME43,42.74,46.55
4,ANTOFAGASTA PLC,1221352,213800MY6QVH4FVLD628,31.46,19.47


We need to create another identifier property for this custom identifier.

In [11]:
# create identifier property for   unique IDs

def create_property_identifier(scope, code, display_name):
    try:
        property_definitions_api.create_property_definition(
            create_property_definition_request=lm.CreatePropertyDefinitionRequest(
                domain="LegalEntity",
                scope=dei_scopeA,
                code=code,
                value_required=None,
                display_name=display_name,
                data_type_id=lm.ResourceId(scope="system", code="string"),
                life_time="Perpetual",
                constraint_style="Identifier"
            )
        )
    except:
        print("Property already exists")

In [12]:
create_property_identifier(dei_scopeA, "Custom_ID", "Custom ID")

Property already exists


The standard LEI from this secondary data source is uploaded using the first identifer property we create so LUSID knows which legal entity is being modified.

In [13]:
# update legal entities with custom identfiers
def update_legal_entity(identifier_property, LEI, internal_ID_property, internal_ID, LE_name):

    legal_entity_request=lm.UpsertLegalEntityRequest(
        identifiers={
            f"LegalEntity/{id_scope}/{identifier_property}": lm.PerpetualProperty(
            key=f"LegalEntity/{id_scope}/{identifier_property}",
            value=lm.PropertyValue(
                label_value=LEI
                )
            ),
            f"LegalEntity/{dei_scopeA}/{internal_ID_property}": lm.PerpetualProperty(
            key=f"LegalEntity/{dei_scopeA}/{internal_ID_property}",
            value=lm.PropertyValue(
                label_value=internal_ID
                )
            ),   
        },
        display_name=LE_name,
        description=LE_name,
        counterparty_risk_information=None,
    )

    legal_entities_api.upsert_legal_entity(legal_entity_request)

In [14]:
dei_sourceA_LEs = []

for i in df_dei_sourceA["Legal Entity Identifier"].index:
    identifier_property="LEI"
    internal_ID_property="Custom_ID"
    dei_sourceA_LEI=df_dei_sourceA["Legal Entity Identifier"][i]
    dei_sourceA_LE_name=df_dei_sourceA["Name"][i]
    internal_ID=str(df_dei_sourceA["Custom Legal Entity Identifier "][i])
    
    LEI = update_legal_entity(identifier_property, dei_sourceA_LEI, internal_ID_property, internal_ID, dei_sourceA_LE_name)
    dei_sourceA_LEs.append(LEI)
    
print(len(dei_sourceA_LEs), "legal entities updated")

90 legal entities updated


In [15]:
# list legal entities with both identifiers
LE_list = legal_entities_api.list_legal_entities(
    id_type_scope=id_scope,
    id_type_code="LEI")

In [16]:
# create dict for renaming df columns

column_rename_mapping = {"display_name":"displayName",
                         "lusid_legal_entity_id":"lusidLegalEntityId",
                         f"identifiers.LegalEntity/{dei_scopeA}/Custom_ID.value.label_value":"customLegalEntityIdentifier",
                         f"identifiers.LegalEntity/{id_scope}/LEI.value.label_value":"legalEntityIdentifier",
                         f"identifiers.LegalEntity/{id_scope}/LEI.key":"legalEntityIdentifierKey",
                         f"identifiers.LegalEntity/{id_scope}/LEI.value.label_value":"legalEntityIdentifier",
                         f"identifiers.LegalEntity/{dei_scopeA}/Custom_ID.key":"customLegalEntityIdentifierKey",
                         f"identifiers.LegalEntity/{dei_scopeA}/Custom_ID.value.label_value":"customLegalEntityIdentifier",
                         f"properties.LegalEntity/{dei_scopeA}/GenderScore.key":"genderScoreKey",
                         f"properties.LegalEntity/{dei_scopeA}/GenderScore.value.metric_value.value":"genderScore",
                         f"properties.LegalEntity/{dei_scopeA}/TotalDEIScore.key":"totalDEIScoreKey",
                         f"properties.LegalEntity/{dei_scopeA}/TotalDEIScore.value.metric_value.value":"totalDEIScore",
                         f"properties.LegalEntity/{dei_scopeB}/RaceEthnicityScore.key":"raceEthnicityScoreKey",
                         f"properties.LegalEntity/{dei_scopeB}/RaceEthnicityScore.value.metric_value.value":"raceEthnicityScore",
                         "values.0.related_entity.display_name":"displayName",
                         "values.0.related_entity.identifiers.1.identifier_value":"instrumentIsin"
                        }


In [17]:
df_LE_list = lusid_response_to_data_frame(LE_list, 
                                          rename_properties=True, 
                                          column_name_mapping=column_rename_mapping)
df_LE_list[["displayName", 
            "description",
            "lusidLegalEntityId", 
            "customLegalEntityIdentifier",
            "legalEntityIdentifier"]].head()

,displayName,description,lusidLegalEntityId,customLegalEntityIdentifier,legalEntityIdentifier
0,BAE SYSTEMS PLC,BAE SYSTEMS PLC,LUID_0004CSVG,132942,8SVCSVKSGDWMW2QHOH83
1,PHOENIX GROUP HOLDINGS PLC,PHOENIX GROUP HOLDINGS PLC,LUID_0004CSVH,1567007,2138001P49OLAEU33T68
2,WPP PLC,WPP PLC,LUID_0004CSVI,NaN,549300LSGBXPYHXGDT93
3,UNITED UTILITIES GROUP PLC,UNITED UTILITIES GROUP PLC,LUID_0004CSVJ,363108,2138002IEYQAOC88ZJ59
4,SMITH & NEPHEW PLC,SMITH & NEPHEW PLC,LUID_0004CSVK,1447521,213800ZTMDN8S67S1H61


## Enhance Legal Entity Information

Now we can create other properties to enhance the legal entity infomation. In this case, we will be updating the legal entities with Diversity, Equity and Inclusion (DEI) data from source A.

In [18]:
# create properties for legal entities

def create_property(scope, dtype, code, display_name):
    try:
        property_definitions_api.create_property_definition(
            create_property_definition_request=lm.CreatePropertyDefinitionRequest(
                domain="LegalEntity",
                scope=scope,
                code=code,
                value_required=None,
                display_name=display_name,
                data_type_id=lm.ResourceId(scope="system", code=dtype),
                life_time="Perpetual",
            )
        )
    except:
        print("Property already exists")

In [19]:
create_property(dei_scopeA, "number", "TotalDEIScore", "Total DEI Score")
create_property(dei_scopeA, "number", "GenderScore", "Gender Score")

Property already exists
Property already exists


In [20]:
# adding properties for DEI data to existing legal entities

def enhance_legal_entity_sourceA(identifier_property, internal_ID_property, LEI, internal_ID, LE_name, TotalDEIScore, GenderScore):

    legal_entity_request= lm.UpsertLegalEntityRequest(
        identifiers={
            f"LegalEntity/{id_scope}/{identifier_property}": lm.PerpetualProperty(
            key=f"LegalEntity/{id_scope}/{identifier_property}",
            value=lm.PropertyValue(
                label_value=LEI
                )
            ), 
        },
        properties={
            f"LegalEntity/{dei_scopeA}/TotalDEIScore":
            lm.PerpetualProperty(
                key=f"LegalEntity/{dei_scopeA}/TotalDEIScore",
                value=lm.PropertyValue(
                    metric_value=lm.MetricValue(TotalDEIScore)
                )
            ),
            f"LegalEntity/{dei_scopeA}/GenderScore":
            lm.PerpetualProperty(
                key=f"LegalEntity/{dei_scopeA}/GenderScore",
                value=lm.PropertyValue(
                    metric_value=lm.MetricValue(GenderScore)
                )
            ),
        },
        display_name=LE_name,
        description=LE_name,
        counterparty_risk_information=None,
    )

    legal_entities_api.upsert_legal_entity(legal_entity_request)

In [21]:
enhanced_LEs = []

for i in df_dei_sourceA["Legal Entity Identifier"].index:
    identifier_property="LEI"
    internal_ID_property="Custom_ID"
    dei_source_LEI=df_dei_sourceA["Legal Entity Identifier"][i]
    dei_source_LE_name=df_dei_sourceA["Name"][i]
    internal_ID=str(df_dei_sourceA["Custom Legal Entity Identifier "][i])
    TotalDEIScore=df_dei_sourceA["Total DEI Score"][i]
    GenderScore=df_dei_sourceA["Total Gender Score"][i]
    
    enhance_LE = enhance_legal_entity_sourceA(identifier_property, internal_ID_property, dei_source_LEI, internal_ID, dei_source_LE_name, TotalDEIScore, GenderScore)
    enhanced_LEs.append(LEI)
    
print(len(enhanced_LEs), "legal entitites updated")

90 legal entitites updated


#### Uploading DEI data from a second source.

In [22]:
df_dei_sourceB = pd.read_csv("data/dei_data_sourceB.csv")
df_dei_sourceB.head()

,Name,Legal Entity Identifier,Total Race/Ethnicity Score
0,IMPERIAL BRANDS PLC,549300DFVPOB67JL3A42,5.55
1,PHOENIX GROUP HOLDINGS PLC,2138001P49OLAEU33T68,1.88
2,ADMIRAL GROUP PLC,213800FGVM7Z9EJB2685,17.70
3,ANGLO AMERICAN PLC,549300S9XF92D1X8ME43,21.08
4,ANTOFAGASTA PLC,213800MY6QVH4FVLD628,19.64


In [23]:
create_property(dei_scopeB, "number", "RaceEthnicityScore", "Race/Ethnicity Score")

Property already exists


In [24]:
# adding properties for DEI data to existing legal entities

def enhance_legal_entity_sourceB(identifier_property, LEI, LE_name, RaceEthnicityScore):

    legal_entity_request=lm.UpsertLegalEntityRequest(
        identifiers={
            f"LegalEntity/{id_scope}/{identifier_property}": 
            lm.PerpetualProperty(
                key=f"LegalEntity/{id_scope}/{identifier_property}",
                value=lm.PropertyValue(
                    label_value=LEI
                )
            ), 
        },
        properties={
            f"LegalEntity/{dei_scopeB}/RaceEthnicityScore":
            lm.PerpetualProperty(
                key=f"LegalEntity/{dei_scopeB}/RaceEthnicityScore",
                value=lm.PropertyValue(
                    metric_value=lm.MetricValue(RaceEthnicityScore)
                )
            ),
        },
        display_name=LE_name,
        description=LE_name,
        counterparty_risk_information=None,
    )

    legal_entities_api.upsert_legal_entity(legal_entity_request)

In [25]:
enhanced_LEs = []

for i in df_dei_sourceB["Legal Entity Identifier"].index:
    identifier_property="LEI"
    dei_sourceB_LEI=df_dei_sourceB["Legal Entity Identifier"][i]
    dei_sourceB_LE_name=df_dei_sourceB["Name"][i]
    RaceEthnicityScore=df_dei_sourceB["Total Race/Ethnicity Score"][i]
    
    enhance_LE = enhance_legal_entity_sourceB(identifier_property, dei_sourceB_LEI, dei_sourceB_LE_name, RaceEthnicityScore)
    enhanced_LEs.append(LEI)
    
print(len(enhanced_LEs), "legal entitites updated")

90 legal entitites updated


## Create Instrument-Legal Entity Relationships

#### Loading DataFrame with ISINs for UK equities and corresponding LEIs
One legal entity can have multiple instruments linked to it in LUSID via instrument identifers. However, in this sample data set, each LEI is linked to only one instrument using the instrument ISIN.

In [26]:
df_inst_ISINs = pd.read_csv("data/Instrument_ISINs.csv")
df_inst_ISINs.head()

,ClientInternal,Name,ISIN,LEI
0,Instrument1,BAE Systems Rg,GB0002634946,8SVCSVKSGDWMW2QHOH83
1,Instrument2,Phoenix Grp Rg,GB00BGXQNP29,2138001P49OLAEU33T68
2,Instrument3,WPP Rg,JE00B8KF9B49,549300LSGBXPYHXGDT93
3,Instrument4,United Utilities Rg,GB00B39J2M42,2138002IEYQAOC88ZJ59
4,Instrument5,Smith & Nephew Rg,GB0009223206,213800ZTMDN8S67S1H61


#### Upserting instruments 
Now we need to create instruments with these ISINs in LUSID to be able to create a relationship with their legal entity.

In [27]:
# instrument field mapping

inst_mapping = {
    "instruments": {
        "identifier_mapping": {"ClientInternal": "ClientInternal", "Isin": "ISIN"},
        "required": {
                        "name": "Name", 
                        "definition.dom_ccy":"$GBP",
                        "definition.asset_class":"$Equities",
                        "definition.instrument_type": "$Equity",
                    },
    },
}

In [28]:
df_inst_unique=df_inst_ISINs[["ISIN",
                            "ClientInternal",
                            "Name",
                             ]].drop_duplicates()
df_inst_unique.head()

,ISIN,ClientInternal,Name
0,GB0002634946,Instrument1,BAE Systems Rg
1,GB00BGXQNP29,Instrument2,Phoenix Grp Rg
2,JE00B8KF9B49,Instrument3,WPP Rg
3,GB00B39J2M42,Instrument4,United Utilities Rg
4,GB0009223206,Instrument5,Smith & Nephew Rg


In [29]:
result=lpt.cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=id_scope,
    data_frame=df_inst_unique,
    mapping_required=inst_mapping["instruments"]["required"],
    mapping_optional={},
    file_type="instrument",
    identifier_mapping=inst_mapping["instruments"]["identifier_mapping"],
    instrument_scope=id_scope
)

succ, failed, errors=format_instruments_response(result)
print(f"number of successful upserts: {len(succ)}")
print(f"number of failed upserts    : {len(failed)}")
print(f"number of errors            : {len(errors)}")

number of successful upserts: 101
number of failed upserts    : 0
number of errors            : 0


### Create relationship between legal entity and instrument

LUSID only accepts unique identifiers when creating relationships. Since ISIN is not a unique identifier, we need a resolver to return the instrument LUID for a given ISIN. Note that this resolver will only work if there is only one instrument with the ISIN in the defined scope. 

In [30]:
# creating resolver to get unique ID for instrument 

def resolver(scope, inst_type, inst_id):
    
    # get the list of LUIDS associated with the instrument
    def no_of_insts(scope, inst_type, inst_id):
        insts_search=search_api.instruments_search(
                                            scope=scope, 
                                            mastered_only=True,
                                            instrument_search_property=[
                                            lm.InstrumentSearchProperty(
                                                key=f"Instrument/default/{inst_type}",
                                                value=inst_id,
                                            )
        ])
        return(insts_search)
    insts_search = no_of_insts(scope, inst_type, inst_id)
    no_insts = len(insts_search[0].mastered_instruments)
    
    if no_insts == 1:
        InstrumentId = insts_search[0].mastered_instruments[0].identifiers["LusidInstrumentId"].value
        
    else:
        InstrumentId=inst_id
        
    return(InstrumentId)

We can test this resolver to show that it brings back the LUID where only one instrument with the given ISIN is found in the instrument scope.

In [31]:
# test for resolver 
resolver(id_scope, "Isin", "GB0002634946")

'LUID_0004CSZ7'

In [32]:
# define legal entity-instrument relationship

def create_relationship_definition(code, display_name, outward_description, inward_description):
    try:
        relationship_definitions_api.create_relationship_definition(
            create_relationship_definition_request=lu.CreateRelationshipDefinitionRequest(
                scope=id_scope,
                code=code,
                source_entity_type="LegalEntity", 
                target_entity_type="Instrument", 
                display_name=display_name, 
                outward_description=outward_description, 
                inward_description=inward_description, 
                life_time="TimeVariant"
            )
        )
        print(f"created relation {id_scope}/{code}")
    except lu.ApiException as e:
        body = json.loads(e.body)
        if body["code"] != 667:  # RelationDefinitionAlreadyExists
            print(body)
        else:
            print(f"relation {id_scope}/{code} already exists")

In [33]:
create_relationship_definition("legalEntity", "Legal Entity-Instrument Relationship", "Issuer of of", "Issued by")

relation legalEntityRelationships/legalEntity already exists


In [34]:
# create relationship

def create_instrument_relationship(code, id_code, id_value, scope, inst_type, inst_id):
    relationships_api.create_relationship(
        
        # the scope/code of the RelationDefinition to be created
        scope=id_scope,
        code=code,
        
        create_relationship_request=lm.CreateRelationshipRequest(
            source_entity_id={
                # the fields the uniquely identify the source entity
                "idTypeScope": id_scope,
                "idTypeCode": id_code,
                "code": id_value
            }, 
            target_entity_id={
                # the fields the uniquely identify the target entity
                "scope": scope,
                "identifierScope": "default",
                "identifierType": "LusidInstrumentId",
                "identifierValue": resolver(scope, inst_type, inst_id)
            },
            effective_from = "2022-01-01T00:00:00+00:00"
            ),
    )

In [35]:
for i in range(len(df_inst_ISINs["ISIN"])):
    create_instrument_relationship("legalEntity", 
                                   "LEI", 
                                   df_inst_ISINs["LEI"][i], 
                                   id_scope, 
                                   "Isin", 
                                   df_inst_ISINs["ISIN"][i])
    
print(f"Created {len(df_inst_ISINs['ISIN'])} relationships between legal entities and instruments.")

Created 101 relationships between legal entities and instruments.


### Return legal entitiy information for a given ISIN using relationships
Now that we have relationships connecting our instruments to their legal entities, we can query the properties on a legal entity using an instrument identifier.

In [36]:
# return the LE-Instrument relationship

def ISIN_to_LEI(instrument_scope, instrument_type, instrument_id):
    inst_rela_result=instruments_api.get_instrument_relationships(
        identifier_type="LusidInstrumentId",
        identifier=resolver(instrument_scope, instrument_type, instrument_id),
        scope=instrument_scope)
    
    inst_rela=  lusid_response_to_data_frame(inst_rela_result, 
                                             rename_properties=True, 
                                             column_name_mapping=column_rename_mapping)
    return(inst_rela)

In [37]:
df_ISIN_to_LEI=ISIN_to_LEI(id_scope, "Isin", "GB0032089863")
df_ISIN_to_LEI

,relationship_definition_id.scope,relationship_definition_id.code,related_entity.entity_type,related_entity.entity_id.idTypeScope,related_entity.entity_id.idTypeCode,related_entity.entity_id.code,related_entity.display_name,related_entity.properties,related_entity.identifiers.0.identifier_scope,related_entity.identifiers.0.identifier_type,related_entity.identifiers.0.identifier_value,related_entity.identifiers.1.identifier_scope,related_entity.identifiers.1.identifier_type,related_entity.identifiers.1.identifier_value,related_entity.identifiers.2.identifier_scope,related_entity.identifiers.2.identifier_type,related_entity.identifiers.2.identifier_value,traversal_direction,traversal_description,effective_from,effective_until
0,legalEntityRelationships,legalEntity,LegalEntity,scope2,Custom_ID,1196655,NEXT PLC,{},scope2,Custom_ID,1196655,legalEntityRelationships,LEI,213800WQD8ECPZP9PH98,deiDataSourceA,Custom_ID,1196655,In,Issued by,2022-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00


In [38]:
# get LE property values using instrument relationship

LE_properties = legal_entities_api.get_legal_entity(id_type_scope=df_ISIN_to_LEI["related_entity.identifiers.1.identifier_scope"][0],
                                    id_type_code=df_ISIN_to_LEI["related_entity.identifiers.1.identifier_type"][0], 
                                    code=str(df_ISIN_to_LEI["related_entity.identifiers.1.identifier_value"][0]),
                                    property_keys=[f"LegalEntity/{dei_scopeA}/TotalDEIScore",
                                                   f"LegalEntity/{dei_scopeA}/GenderScore"] )

In [39]:
properties = lusid_response_to_data_frame(LE_properties)
properties_transposed = properties.transpose(copy=True)
df_LE_properties = pd.DataFrame(properties_transposed[["display_name",
                                          "description",
                                          "lusid_legal_entity_id",
                                          f"identifiers.LegalEntity/{dei_scopeA}/Custom_ID.key",
                                          f"identifiers.LegalEntity/{dei_scopeA}/Custom_ID.value.label_value",
                                          f"identifiers.LegalEntity/{id_scope}/LEI.key",
                                          f"identifiers.LegalEntity/{id_scope}/LEI.value.label_value",
                                          f"properties.LegalEntity/{dei_scopeA}/GenderScore.key",
                                          f"properties.LegalEntity/{dei_scopeA}/GenderScore.value.metric_value.value",
                                          f"properties.LegalEntity/{dei_scopeA}/TotalDEIScore.key",
                                          f"properties.LegalEntity/{dei_scopeA}/TotalDEIScore.value.metric_value.value"
                                         ]])

df_LE_properties.rename(columns=column_rename_mapping, inplace=True)
df_LE_properties

,displayName,description,lusidLegalEntityId,customLegalEntityIdentifierKey,customLegalEntityIdentifier,legalEntityIdentifierKey,legalEntityIdentifier,genderScoreKey,genderScore,totalDEIScoreKey,totalDEIScore
response_values,NEXT PLC,NEXT PLC,LUID_0004CSX6,LegalEntity/deiDataSourceA/Custom_ID,1196655,LegalEntity/legalEntityRelationships/LEI,213800WQD8ECPZP9PH98,LegalEntity/deiDataSourceA/GenderScore,48.81,LegalEntity/deiDataSourceA/TotalDEIScore,41.03


## Filter an instrument universe based on LE DEI data

Now that we have instruments linked to legal entities, we can create a universe of instruments based on the diversity data attached to their corresponding legal entities.

In [40]:
# listing all legal entities in scope that meet DEI criteria

LEs=legal_entities_api.list_legal_entities(id_type_scope=id_scope,
                                             id_type_code="LEI",
                                             property_keys=[f"LegalEntity/{dei_scopeA}/TotalDEIScore",
                                                           f"LegalEntity/{dei_scopeB}/RaceEthnicityScore"],
                                             
                                             # filter by best DEI scores
                                             filter=f"properties[LegalEntity/{dei_scopeA}/TotalDEIScore] gt 45 and properties[LegalEntity/{dei_scopeB}/RaceEthnicityScore] gt 17",
                                             limit=110)

df_LEs = lusid_response_to_data_frame(LEs, column_name_mapping=column_rename_mapping)
df_filtered_LEs = df_LEs[["displayName",
                          "description",
                          "lusidLegalEntityId",
                          "customLegalEntityIdentifierKey",
                          "customLegalEntityIdentifier",
                          "legalEntityIdentifierKey",
                          "legalEntityIdentifier",
                          "totalDEIScoreKey",
                          "totalDEIScore",
                          "raceEthnicityScoreKey",
                          "raceEthnicityScore"
                         ]]
df_filtered_LEs

,displayName,description,lusidLegalEntityId,customLegalEntityIdentifierKey,customLegalEntityIdentifier,legalEntityIdentifierKey,legalEntityIdentifier,totalDEIScoreKey,totalDEIScore,raceEthnicityScoreKey,raceEthnicityScore
0,HALMA PUBLIC LIMITED COMPANY,HALMA PUBLIC LIMITED COMPANY,LUID_0004CSVO,LegalEntity/deiDataSourceA/Custom_ID,628845,LegalEntity/legalEntityRelationships/LEI,2138007FRGLUR9KGBT40,LegalEntity/deiDataSourceA/TotalDEIScore,48.77,LegalEntity/deiDataSourceB/RaceEthnicityScore,18.47
1,BP PLC,BP PLC,LUID_0004CSVT,LegalEntity/deiDataSourceA/Custom_ID,1015609,LegalEntity/legalEntityRelationships/LEI,213800LH1BZH3DI6G760,LegalEntity/deiDataSourceA/TotalDEIScore,46.96,LegalEntity/deiDataSourceB/RaceEthnicityScore,32.74
2,BURBERRY GROUP PLC,BURBERRY GROUP PLC,LUID_0004CSVV,LegalEntity/deiDataSourceA/Custom_ID,1234613,LegalEntity/legalEntityRelationships/LEI,213800PE1KEFCNFR1R50,LegalEntity/deiDataSourceA/TotalDEIScore,47.60,LegalEntity/deiDataSourceB/RaceEthnicityScore,18.46
3,ROYAL DUTCH SHELL PLC,ROYAL DUTCH SHELL PLC,LUID_0004CSW3,LegalEntity/deiDataSourceA/Custom_ID,875744,LegalEntity/legalEntityRelationships/LEI,21380068P1DRHMJ8KU70,LegalEntity/deiDataSourceA/TotalDEIScore,47.28,LegalEntity/deiDataSourceB/RaceEthnicityScore,18.46
4,SEVERN TRENT PLC,SEVERN TRENT PLC,LUID_0004CSW6,LegalEntity/deiDataSourceA/Custom_ID,122294,LegalEntity/legalEntityRelationships/LEI,213800RPBXRETY4A4C59,LegalEntity/deiDataSourceA/TotalDEIScore,45.62,LegalEntity/deiDataSourceB/RaceEthnicityScore,17.56
5,ITV PLC,ITV PLC,LUID_0004CSWC,LegalEntity/deiDataSourceA/Custom_ID,1343395,LegalEntity/legalEntityRelationships/LEI,ZLECI7ED2QMWFGYCXZ59,LegalEntity/deiDataSourceA/TotalDEIScore,45.25,LegalEntity/deiDataSourceB/RaceEthnicityScore,39.38
6,ASTRAZENECA PLC,ASTRAZENECA PLC,LUID_0004CSWR,LegalEntity/deiDataSourceA/Custom_ID,1329415,LegalEntity/legalEntityRelationships/LEI,PY6ZZQWO2IZFZC3IOL08,LegalEntity/deiDataSourceA/TotalDEIScore,49.11,LegalEntity/deiDataSourceB/RaceEthnicityScore,20.53
7,RIGHTMOVE PLC,RIGHTMOVE PLC,LUID_0004CSXA,LegalEntity/deiDataSourceA/Custom_ID,1563933,LegalEntity/legalEntityRelationships/LEI,2138001JXGCFKBXYB828,LegalEntity/deiDataSourceA/TotalDEIScore,45.16,LegalEntity/deiDataSourceB/RaceEthnicityScore,18.76
8,UNILEVER PLC,UNILEVER PLC,LUID_0004CSXC,LegalEntity/deiDataSourceA/Custom_ID,916465,LegalEntity/legalEntityRelationships/LEI,549300MKFYEKVRWML317,LegalEntity/deiDataSourceA/TotalDEIScore,45.04,LegalEntity/deiDataSourceB/RaceEthnicityScore,20.46
9,PEARSON PLC,PEARSON PLC,LUID_0004CSXF,LegalEntity/deiDataSourceA/Custom_ID,1089313,LegalEntity/legalEntityRelationships/LEI,2138004JBXWWJKIURC57,LegalEntity/deiDataSourceA/TotalDEIScore,50.13,LegalEntity/deiDataSourceB/RaceEthnicityScore,32.76


In [41]:
# getting instruments related to filtered legal entity list

relationships = []
LEIs = []

for i in df_filtered_LEs.index:
    inst_relationship = legal_entities_api.get_legal_entity_relationships(id_type_scope=id_scope,
                                                  id_type_code="LEI",
                                                  code= df_LEs["legalEntityIdentifier"][i])
    relationships.append(inst_relationship)
    LEIs.append(df_LEs["legalEntityIdentifier"][i])

In [42]:
# add the legal entity identifiers to the DataFrame
df_relationships = lusid_response_to_data_frame(relationships, 
                                                rename_properties=True,
                                                column_name_mapping=column_rename_mapping)

df_relationships["legalEntityIdentifier"]=LEIs
df_relationships.head()

,values.0.relationship_definition_id.scope,values.0.relationship_definition_id.code,values.0.related_entity.entity_type,values.0.related_entity.entity_id.identifierScope,values.0.related_entity.entity_id.identifierType,values.0.related_entity.entity_id.identifierValue,values.0.related_entity.entity_id.scope,displayName,values.0.related_entity.properties,values.0.related_entity.scope,values.0.related_entity.lusid_unique_id.type,values.0.related_entity.lusid_unique_id.value,values.0.related_entity.identifiers.0.identifier_scope,values.0.related_entity.identifiers.0.identifier_type,values.0.related_entity.identifiers.0.identifier_value,values.0.related_entity.identifiers.1.identifier_scope,values.0.related_entity.identifiers.1.identifier_type,instrumentIsin,values.0.related_entity.href,values.0.traversal_direction,values.0.traversal_description,values.0.effective_from,values.0.effective_until,href,links.0.relation,links.0.href,links.0.description,links.0.method,values.1.relationship_definition_id.scope,values.1.relationship_definition_id.code,values.1.related_entity.entity_type,values.1.related_entity.entity_id.identifierScope,values.1.related_entity.entity_id.identifierType,values.1.related_entity.entity_id.identifierValue,values.1.related_entity.entity_id.scope,values.1.related_entity.display_name,values.1.related_entity.properties,values.1.related_entity.scope,values.1.related_entity.lusid_unique_id.type,values.1.related_entity.lusid_unique_id.value,values.1.related_entity.identifiers.0.identifier_scope,values.1.related_entity.identifiers.0.identifier_type,values.1.related_entity.identifiers.0.identifier_value,values.1.related_entity.identifiers.1.identifier_scope,values.1.related_entity.identifiers.1.identifier_type,values.1.related_entity.identifiers.1.identifier_value,values.1.related_entity.href,values.1.traversal_direction,values.1.traversal_description,values.1.effective_from,values.1.effective_until,legalEntityIdentifier
0,legalEntityRelationships,legalEntity,Instrument,default,ClientInternal,Instrument9,legalEntityRelationships,Halma Rg,{},legalEntityRelationships,LusidInstrumentId,LUID_0004CSYH,default,ClientInternal,Instrument9,default,Isin,GB0004052071,https://fbn-ci.lusid.com/api/api/instruments/C...,Out,Issuer of of,2022-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,https://fbn-ci.lusid.com/api/api/legalentities...,RequestLogs,http://fbn-ci.lusid.com/app/insights/logs/0HMM...,A link to the LUSID Insights website showing a...,GET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2138007FRGLUR9KGBT40
1,legalEntityRelationships,legalEntity,Instrument,default,ClientInternal,Instrument14,legalEntityRelationships,BP Rg,{},legalEntityRelationships,LusidInstrumentId,LUID_0004CSYO,default,ClientInternal,Instrument14,default,Isin,GB0007980591,https://fbn-ci.lusid.com/api/api/instruments/C...,Out,Issuer of of,2022-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,https://fbn-ci.lusid.com/api/api/legalentities...,RequestLogs,http://fbn-ci.lusid.com/app/insights/logs/0HMM...,A link to the LUSID Insights website showing a...,GET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,213800LH1BZH3DI6G760
2,legalEntityRelationships,legalEntity,Instrument,default,ClientInternal,Instrument16,legalEntityRelationships,Burberry Group Rg,{},legalEntityRelationships,LusidInstrumentId,LUID_0004CT0F,default,ClientInternal,Instrument16,default,Isin,GB0031743007,https://fbn-ci.lusid.com/api/api/instruments/C...,Out,Issuer of of,2022-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,https://fbn-ci.lusid.com/api/api/legalentities...,RequestLogs,http://fbn-ci.lusid.com/app/insights/logs/0HMM...,A link to the LUSID Insights website showing a...,GET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,213800PE1KEFCNFR1R50
3,legalEntityRelationships,legalEntity,Instrument,default,ClientInternal,Instrument76,legalEn

In [43]:
# filter to ensure only LE-instrument relationships
df_inst_relationships = df_relationships[df_relationships["values.0.related_entity.entity_type"] == "Instrument"]
df_inst_relationships.rename(columns = {"displayName":"instrumentDisplayName"}, inplace=True)
df_inst_relationships = df_inst_relationships[["instrumentDisplayName",
                                               "instrumentIsin",
                                               "legalEntityIdentifier"]]
df_inst_relationships.head()

,instrumentDisplayName,instrumentIsin,legalEntityIdentifier
0,Halma Rg,GB0004052071,2138007FRGLUR9KGBT40
1,BP Rg,GB0007980591,213800LH1BZH3DI6G760
2,Burberry Group Rg,GB0031743007,213800PE1KEFCNFR1R50
3,Royal Dutch Shell-A,GB00B03MLX29,21380068P1DRHMJ8KU70
4,Severn Trent Rg,GB00B1FH8J72,213800RPBXRETY4A4C59


Now we can merge DataFrames so you can see the universe of ISINs meeting the DEI score criteria along with the LEI of their legal entity and the Total DEI Score.

In [44]:
merged_df = df_inst_relationships.merge(df_filtered_LEs, left_on="legalEntityIdentifier", right_on="legalEntityIdentifier")
filtered_universe = pd.DataFrame(merged_df[["instrumentDisplayName",
                                            "instrumentIsin",
                                            "legalEntityIdentifier",
                                            "totalDEIScore",
                                            "raceEthnicityScore"]])
filtered_universe

,instrumentDisplayName,instrumentIsin,legalEntityIdentifier,totalDEIScore,raceEthnicityScore
0,Halma Rg,GB0004052071,2138007FRGLUR9KGBT40,48.77,18.47
1,BP Rg,GB0007980591,213800LH1BZH3DI6G760,46.96,32.74
2,Burberry Group Rg,GB0031743007,213800PE1KEFCNFR1R50,47.60,18.46
3,Royal Dutch Shell-A,GB00B03MLX29,21380068P1DRHMJ8KU70,47.28,18.46
4,Severn Trent Rg,GB00B1FH8J72,213800RPBXRETY4A4C59,45.62,17.56
5,ITV Rg,GB0033986497,ZLECI7ED2QMWFGYCXZ59,45.25,39.38
6,AstraZeneca Rg,GB0009895292,PY6ZZQWO2IZFZC3IOL08,49.11,20.53
7,Rightmove Rg,GB00BGDT3G23,2138001JXGCFKBXYB828,45.16,18.76
8,Unilever Rg,GB00B10RZP78,549300MKFYEKVRWML317,45.04,20.46
9,Pearson Rg,GB0006776081,2138004JBXWWJKIURC57,50.13,32.76
